In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('btceUSD_1-min_data_2012-01-01_to_2017-05-31.csv')

In [3]:
df = df.iloc[620140:,:].copy()

In [4]:
df['Close'] = df['Close'].ffill()

In [5]:
df_close = pd.Series(df['Close'])

In [6]:
df_close.to_csv('close_3.csv')

In [7]:
df.shape

(2131454, 8)

In [8]:
pd.isnull(df).sum()

Timestamp                 0
Open                 336860
High                 336860
Low                  336860
Close                     0
Volume_(BTC)         336860
Volume_(Currency)    336860
Weighted_Price       336860
dtype: int64

In [9]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import time

Using TensorFlow backend.


In [10]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [11]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [12]:
dataframe = pd.read_csv('close_3.csv', usecols=[1], engine='python', skipfooter=3)
dataset = dataframe.values
dataset = dataset.astype('float32')

In [13]:
dataset.shape

(2131450, 1)

In [14]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [15]:
# split into train and test sets
train_size = int(len(dataset) * 0.90)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
# reshape into X=t and Y=t+1

In [16]:
look_back = 30
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
#trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
#testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
testX.shape

(213114, 30)

In [17]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [18]:
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=15, batch_size=3000, verbose=2)

Epoch 1/15
10s - loss: 1.7236e-04
Epoch 2/15
10s - loss: 1.1612e-06
Epoch 3/15
11s - loss: 1.1000e-06
Epoch 4/15
8s - loss: 1.0440e-06
Epoch 5/15
11s - loss: 9.7310e-07
Epoch 6/15
10s - loss: 8.8875e-07
Epoch 7/15
8s - loss: 7.9252e-07
Epoch 8/15
10s - loss: 6.8875e-07
Epoch 9/15
10s - loss: 5.8864e-07
Epoch 10/15
8s - loss: 5.1336e-07
Epoch 11/15
9s - loss: 4.8136e-07
Epoch 12/15
9s - loss: 4.7319e-07
Epoch 13/15
8s - loss: 4.5668e-07
Epoch 14/15
8s - loss: 4.5328e-07
Epoch 15/15
8s - loss: 4.3623e-07


In [19]:
from keras.models import load_model

model.save('my_model_btce.h5')  # creates a HDF5 file 'my_model.h5'
#del model